In [7]:
import asyncio
import random
import time

In [ ]:
# worker coro that takes queue as input
async def worker(name, queue: asyncio.Queue):
    while True:
        # step 1: Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # step 2: process the work item
        await asyncio.sleep(sleep_for)

        # step 3: notify done and log
        queue.task_done()
        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # work items 
    total_sleep_times = []
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_times.append(sleep_for)

    # queue
    queue = asyncio.Queue()
    for sleep_for in total_sleep_times:
        queue.put_nowait(sleep_for)

    # worker tasks
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # exec
    started_at = time.monotonic()

    await queue.join()
    for task in tasks:
        task.cancel()# to cancel infinite tasks, schedule cancelled error to be thrown at next await
    await asyncio.gather(*tasks, return_exceptions=True) # wait until all tasks are cancelled

    total_slept_for = time.monotonic() - started_at

    # log
    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {sum(total_sleep_times):.2f} seconds')


In [ ]:
await main()

worker-2 has slept for 0.06 seconds
worker-1 has slept for 0.27 seconds
worker-2 has slept for 0.41 seconds
worker-0 has slept for 0.59 seconds
worker-1 has slept for 0.98 seconds
worker-0 has slept for 0.72 seconds
worker-2 has slept for 0.93 seconds
worker-1 has slept for 0.18 seconds
worker-0 has slept for 0.39 seconds
worker-0 has slept for 0.10 seconds
worker-0 has slept for 0.06 seconds
worker-2 has slept for 0.62 seconds
worker-1 has slept for 0.70 seconds
worker-2 has slept for 0.16 seconds
worker-1 has slept for 0.10 seconds
worker-2 has slept for 0.19 seconds
worker-0 has slept for 0.62 seconds
worker-0 has slept for 0.23 seconds
worker-2 has slept for 0.42 seconds
worker-1 has slept for 0.60 seconds
====
3 workers slept in parallel for 2.83 seconds
total expected sleep time: 8.32 seconds
